# Min/Max of Functions in Python

In [13]:
import scipy
print(scipy.__file__)

/opt/homebrew/Caskroom/miniforge/base/envs/lighthouse/lib/python3.8/site-packages/scipy/__init__.py


In [14]:
from pathlib import Path
import numpy as np
from scipy.cluster.vq import whiten, kmeans, vq

In [16]:
data = Path('SMSSpamCollections').read_text()
data = data.strip()
data = data.split("\n")

In [29]:
print(type(data), len(data), 'objects')

<class 'list'> 5574 objects


In [26]:
digit_counts = np.empty((len(data), 2), dtype=int)

In the dataset, each message has one of two labels:
1. 'ham' for legitimate messages
2. 'spam' for spam messages

The full text message is associated with each label. When you scan through the data, you might notice that spam messages tend to have a lot of numeric digits in them. They often include a phone number or prize winnings. Let's predict whether or not a message is spam based on the number of digits in the message. To do this, you'll **cluster** the data into three groups based on the number of digits that appear in the message:
1. **Not Spam**: Messages with the smallest number of digits are predicted not to be spam.
2. **Unknown**: Messages with an intermediate number of digits are unknown and need to be processed by more advanced algorithms.
3. **Spam**: Messages with the highest number of digits are predicted to be spam.

In [37]:
for i, line in enumerate(data):
    case, message = line.split("\t") # splits by tab
    num_digits = sum(c.isdigit() for c in message) # checks for number of digits and counts them
    digit_counts[i, 0] = 0 if case == "ham" else 1 # binary signature to designate message as spam or ham
    digit_counts[i, 1] = num_digits # place digit count in respective row along with spam/ham designation

There are 5,574 observation (or individual messages) in the dataset with 2 features:
1. **The number of digits** in a text message
2. **The number of times** that number of digits appears in the whole dataset

In [38]:
unique_counts = np.unique(digit_counts[:, 1], return_counts=True) # collects unique set of digits and returns a value count
unique_counts = np.transpose(np.vstack(unique_counts)) # transform the 1xN array into 2xN (vector) and transpose into Nx2

`np.unique()` takes an array as the first argument and returns another array with the unique elements from the argument. It also takes several optional arguments. Here, you use `return_counts=True` to instruct `np.unique()` to also return an array with the number of times each unique element is present in the input array.

Each row in `unique_counts` now has two elements:
1. **The number of digits** in a message
2. **The number of messages** that had that number of digits

In [42]:
whitened_counts = whiten(unique_counts) # normalize features for unit variance to improve results from kmeans()
codebook, _ = kmeans(whitened_counts, 3) # 3 clusters: 'ham', 'spam', 'unknown'

In [43]:
codes, _ = vq(whitened_counts, codebook)

vq() assigns codes from the codebook to each observation.

In [46]:
ham_code = codes[0] # ham should have least digits
spam_code = codes[-1] # spam should have most
unknown_code = list(set(range(3)) ^ set((ham_code, spam_code)))[0] # unknown are somewhere in the middle

In [47]:
print("definitely ham:", unique_counts[codes == ham_code][-1])
print("definitely spam:", unique_counts[codes == spam_code][-1])
print("unknown:", unique_counts[codes == unknown_code][-1])

definitely ham: [   0 4110]
definitely spam: [47  1]
unknown: [20 18]


Process to measure accuracy of predictions.

In [50]:
digits = digit_counts[:, 1]
predicted_hams = digits == 0
predicted_spams = digits > 20
predicted_unknowns = np.logical_and(digits > 0, digits <= 20)

In [52]:
spam_cluster = digit_counts[predicted_spams]
ham_cluster = digit_counts[predicted_hams]
unk_cluster = digit_counts[predicted_unknowns]

In [53]:
print("hams:", np.unique(ham_cluster[:, 0], return_counts=True))
print("spams:", np.unique(spam_cluster[:, 0], return_counts=True))
print("unknowns:", np.unique(unk_cluster[:, 0], return_counts=True))

hams: (array([0, 1]), array([4071,   39]))
spams: (array([0, 1]), array([  1, 232]))
unknowns: (array([0, 1]), array([755, 476]))


### Using the Optimize Module in SciPy

- `minimize_scalar()` and `minimize()` to minimize a function of one variable and many variables, respectively
- `curve_fit()` to fit a function to a set of data
- `root_scalar()` and `root()` to find the zeros of a function of one variable and many variables, respectively
- `linprog()` to minimize a linear objective function with linear inequality and equality constraints

In practice, all of these functions are performing *optimization* of one sort or another. In this section, you'll learn about the two minimization functions.

### Minimizaing a Function With One Variable

A mathematical function that accepts one number of results in one output is called a **scalar function**.

In [54]:
from scipy.optimize import minimize_scalar

def objective_function(x):
    return 3 * x ** 4 - 2 * x + 1

In [55]:
res = minimize_scalar(objective_function)

The output of `minimize_scalar()` is an instance of `OptimizeResult`. This class collects together many of the relevant details from teh optimizer's run, including whether or not the optimization was successful and, if succesful, what the final result was.

In [56]:
res

     fun: 0.17451818777634331
 message: '\nOptimization terminated successfully;\nThe returned value satisfies the termination criteria\n(using xtol = 1.48e-08 )'
    nfev: 16
     nit: 12
 success: True
       x: 0.5503212087491959

NOTE: Not every function has a minimum And some functions have serveral minima. In these cases, `minimize_scalar()` is not guaranteed to find a global minimum of the function. However, a method keyword argument can be used to specify the solver for certain optimizations.
1. `brent` is an implementation of **Brent's algorithm**. This is the default.
2. `golden` is the implementation of the **golden-section search**.
3. `bounded` is a bounded implementation of **Brent's algorithm**. It's useful to limit the search region when the minimum is in a known range.

In [57]:
def objective_function(x):
    return x ** 4 - x ** 2

In [58]:
res = minimize_scalar(objective_function)

In [64]:
res

     fun: -0.24999999999999994
 message: '\nOptimization terminated successfully;\nThe returned value satisfies the termination criteria\n(using xtol = 1.48e-08 )'
    nfev: 15
     nit: 11
 success: True
       x: 0.7071067853059209

In [65]:
res_bracket = minimize_scalar(objective_function, bracket=(-1, 0))
# the bracket defines the values between which the result should output in case there exists more
# than one solution    

In [66]:
res_bracket

     fun: -0.24999999999999997
 message: '\nOptimization terminated successfully;\nThe returned value satisfies the termination criteria\n(using xtol = 1.48e-08 )'
    nfev: 17
     nit: 13
 success: True
       x: 0.7071067809244586

### Minimizing a Function with Many Variables

`minimize()` can handle **multivariate** inputs and outputs and has more complicated optimization algorithms to be able to handle this. In addition, `minimize()` can handle **constraints** on the solution to your problem.
1. `LinearConstraint` is a solution that is constrained by taking the inner product of the solution 'x' values with a user-input array and comparing the result to a lower and upper bound.
2. `NonlinearConstraint` is a solution that is constrained by applying a user-supplied function to the solution 'x' values and comparing the return value with a lower and upper bound.
3. `Bounds` is the solution 'x' values constrained to lie between a lower and upper bound.

In [67]:
import numpy as np
from scipy.optimize import minimize, LinearConstraint

n_buyers = 10
n_shares = 15

In [68]:
np.random.seed(10)
prices = np.random.random(n_buyers)
money_available = np.random.randint(1, 4, n_buyers)

In [69]:
n_shares_per_buyer = money_available / prices
print(prices, money_available, n_shares_per_buyer, sep="\n")

[0.77132064 0.02075195 0.63364823 0.74880388 0.49850701 0.22479665
 0.19806286 0.76053071 0.16911084 0.08833981]
[1 1 1 3 1 3 3 2 1 1]
[ 1.29647768 48.18824404  1.57816269  4.00638948  2.00598984 13.34539487
 15.14670609  2.62974258  5.91328161 11.3199242 ]


In [70]:
constraint = LinearConstraint(np.ones(n_buyers), lb=n_shares, ub=n_shares)

In [71]:
bounds = [(0, n) for n in n_shares_per_buyer]

In [72]:
def objective_function(x, prices):
    return -x.dot(prices)

In [73]:
res = minimize(
    objective_function,
    x0=10 * np.random.random(n_buyers),
    args=(prices,),
    constraints=constraint,
    bounds=bounds,
)

In [74]:
res

     fun: -8.78302015708768
     jac: array([-0.7713207 , -0.02075195, -0.63364828, -0.74880397, -0.49850702,
       -0.22479665, -0.1980629 , -0.76053071, -0.16911089, -0.08833981])
 message: 'Optimization terminated successfully'
    nfev: 187
     nit: 17
    njev: 17
  status: 0
 success: True
       x: array([1.29647768e+00, 3.73026111e-14, 1.57816269e+00, 4.00638948e+00,
       2.00598984e+00, 3.48323773e+00, 6.66133815e-16, 2.62974258e+00,
       2.79628716e-15, 5.05103555e-15])

In [75]:
print("The total number of shares is:", sum(res.x))
print("Leftover money for each buyer:", money_available - res.x * prices)

The total number of shares is: 14.999999999999996
Leftover money for each buyer: [3.66373598e-15 1.00000000e+00 1.77635684e-15 3.55271368e-15
 2.10942375e-15 2.21697984e+00 3.00000000e+00 4.88498131e-15
 1.00000000e+00 1.00000000e+00]
